In [ ]:
from googleAPI import *
from pymongo import MongoClient
from crawling import ProgressBar
from pprint import pprint

In [ ]:
client = MongoClient(
    "mongodb+srv://cluster0.c6ccx.mongodb.net",
    authsource="$external",
    authmechanism="MONGODB-X509",
    tls=True,
    tlsCertificateKeyFile="../../../mongodb.pem",
)
db = client["youtube"]


In [ ]:
with open("../../../apikey.txt", encoding="utf8") as f:
    apikey = f.readline().strip()


In [ ]:
import datetime

# 1970-01-01T00:00:00Z
def datetime_to_string(datetime: datetime.datetime):
    return datetime.strftime("%Y-%m-%dT%H:%M:%SZ")


def string_to_datetime(str):
    return datetime.datetime().strptime(str, "%Y-%m-%dT%H:%M:%SZ")


# search with keyword(costs to much(100 times much than other))
def get_searched_video(query):
    init_query = {
        "key": apikey,
        "maxResults": 50,
        "part": "id, snippet",
        "type": "video",
        "regionCode": "KR",
        "safeSearch": "strict"
    }
    for key in query:
        init_query[key] = query[key]
    return get_youtube_api_json("search", init_query)


In [ ]:
# get top view count video of korea from search
videoSearch = db["videoSearch"]
start_date = datetime_to_string(datetime.datetime(2022, 2, 25))
end_date = datetime_to_string(datetime.datetime(2022, 2, 26))
init_query = {"order": "viewCount", "publishedAfter": start_date, "publishedBefore": end_date}
total_search_data = []
nextPageToken = ""
for i in range(20):
    init_query["pageToken"] = nextPageToken
    search_data = get_searched_video(init_query)
    total_search_data.append(search_data)
    nextPageToken = search_data.get("nextPageToken")
    if nextPageToken is None:
        print(search_data)
        break
videoSearch.insert_many(total_search_data)

In [ ]:
videoSearch.find()

In [ ]:
# get category data of korea
videoCategories = db["videoCategories"]
categoryData = get_youtube_api_json("videoCategories", {"key": apikey, "part": "snippet, id", "regionCode": "KR"})[
    "items"
]
videoCategories.insert_many(categoryData)

In [ ]:
list(videoCategories.find())


In [ ]:
def get_populer_video(query):
    init_query = {
        "key": apikey,
        "maxResults": 50,
        "part": "snippet, contentDetails, status, statistics, player, topicDetails, recordingDetails",
        "chart": "mostPopular",
        "regionCode": "KR",
    }
    for key in query:
        init_query[key] = query[key]
    return get_youtube_api_json("videos", init_query)

def get_video(query):
    init_query = {
        "key": apikey,
        "maxResults": 50,
        "part": "snippet, contentDetails, status, statistics, player, topicDetails, recordingDetails",
        "regionCode": "KR",
    }
    for key in query:
        init_query[key] = query[key]
    return get_youtube_api_json("videos", init_query)

In [ ]:
# videoSearch = db["videoSearch"]
# rawVideoSearch = db["rawVideoSearch"]
# rawVideoSearch.insert_many(videoSearch.find())
# videoSearch.drop()

In [ ]:
import re

hangul = re.compile("[ㄱ-ㅎ가-힣]+")
rawVideoSearch = db["rawVideoSearch"]
videoSearch = db["videoSearch"]
totalPreData = []
for datas in rawVideoSearch.find():
    for data in datas["items"]:
        try:
            preData = get_video({"id":data["id"]["videoId"]})["items"][0]
        except:
            continue
        if len(hangul.findall(preData["snippet"]["title"])) > 0:
            print(preData["snippet"]["title"])
            totalPreData.append(preData)
videoSearch.insert_one({"searchDate":datetime.datetime.now(),"info":totalPreData})

In [ ]:
data = list(videoSearch.find())
pre_data = []
for i in data:
    for j in i["info"]:
        pre_data.append((int(j["statistics"]["viewCount"]), j["snippet"]["title"].strip()))
pre_data.sort(reverse=True)
pre_data

In [ ]:
for categoryData in videoCategories.find():
    categoryId = categoryData["id"]
    print(categoryId)

In [ ]:
get_video({"id":"8Syy24YLOQw"})

In [ ]:
videoFavorite = db["videoFavorite"]
totalData = []
for categoryData in videoCategories.find():
    categoryId = categoryData["id"]
    nextPageToken = ""
    for i in range(20):
        favoriteData = get_populer_video({"videoCategoryId": categoryId, "pageToken":nextPageToken})
        nextPageToken = favoriteData.get("nextPageToken")
        totalData.append({"searchDate":datetime.datetime.now(), "searchCategory":categoryId,"info":favoriteData})
        if nextPageToken is None:
            break
videoFavorite.insert_many(totalData)

In [ ]:
init_data = get_populer_video()
totalResults = init_data["pageInfo"]["totalResults"]
print(totalResults)
# nextPageToken = init_data.get("nextPageToken")
# populer_video_data = [init_data]
# for i in range(5):
#     print(nextPageToken)
#     data = get_populer_video(nextPageToken)
#     populer_video_data.append(data)
#     nextPageToken = data.get("nextPageToken")
#     if nextPageToken is None:
#         break
# pprint(populer_video_data)


In [ ]:
get_youtube_api_json(
    "commentThreads",
    {
        "key": apikey,
        "part": "id, replies, snippet",
        "maxResults": 100,
        "textFormat": "plainText",
        "videoId": "zWhqy34zvVI",
        "order": "relevance",
        # "searchTerms":"차준환"
    },
)
